---
##### Goal of Script: Pull data from a website 

How do candidates identify potential companies during a job search?

One method is to read online articles such as Fortune's "Best Workplaces for Millennials" ([link](https://www.greatplacetowork.com/best-workplaces/Millennials/2016)).I created a script that retrieves (i.e. web scrapes) company information directly from the website. 

---
##### Step 1: Define data
Fortune publishes a "Best Workplaces for Millennials" article every year. There are currently 5 articles available online (2016 to 2021) and each article has the same html structure. My script retrieves information for all 5 years. 

Note: The articles from 2019 - 2021 separate the list into two categories: Large and Small/Medium ([link](https://www.greatplacetowork.com/best-workplaces/Millennials/2019)). However, python sees this as the same webpage. 

---
##### Step 2: Define assumptions

* Fortune will continue to use the same html structure in the future
* Fortune will continue to allow web scraping i.e. the site isn't blocked 
* A company can only occur once on the Fortune list every year

---
##### Step 3: Create the script

The script combines all the data together from the online articles into a single object called a dataframe (companies_df). The dataframe can then be manipulated in any way. 

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd

site = 'https://www.greatplacetowork.com/best-workplaces/millennials' 
current_year = date.today().year
companies = []
error_msg = []

for year in range(2016,current_year + 1): #loop though all years; separate url for each year
    
    url = site + "/" + str(year)
    page = requests.get(url) #request access to website

    if str(page.status_code)[0] == '2':
        soup=BeautifulSoup(page.text,'html.parser')
        div = soup.find_all('div',{"class":"col-md-5 col-xs-12 company-text"}) #common class that identifies the info for each company

        for company in div: 
            if len(company.find_all("ul")) < 3:  #skip companies that don't have complete info.
                continue
            else:  
                companies.append(
                    {"Name": company.a['title'],
                     "Industry": company.find("ul",{"class":"industry fa-ul"}).li.i.next_sibling,
                     "Location": company.find("ul",{"class":"location fa-ul"}).li.i.next_sibling, 
                     "Year": year})
    else:
        error_msg.append("The webpage {} gave the error {}".format(url,page.status_code)) #error if webpage denies request
        
if len(error_msg) != 0:
    print(error_msg)
    
#HTML Notes:
#div[0].li.i.attrs #can get attributes of a nested tag
#div[0].li.i.next_sibling #Since industry is listed first in li tag sequence, this returns industry dict

---
##### Final Product: Dataframe with 4 columns

The companies_df gives the company name, the location of the company, the year the company was on the Fortune list, and the industry.The data is futher explored and then exported as an excel file. 

In [2]:
companies_df = pd.DataFrame(companies) #Create dataframe
companies_df.head() #show first 5 entries of dataframe

,Industry,Location,Name,Year
0,Advertising & Marketing,"New York, NY, US",Elite SEM,2016
1,Financial Services & Insurance,"Detroit, MI, US",Rocket Companies,2016
2,Professional Services,"Raleigh, NC, US","Kimley-Horn and Associates, Inc",2016
3,Professional Services,"Seattle, WA, US",Point B Inc.,2016
4,Information Technology,"WESTON, FL, US",ULTIMATE SOFTWARE,2016


In [3]:
#What are the top two companies for each industry? Freq is the number of times they occurred on the Fortune list

counts_df = companies_df.drop('Location',1).groupby(['Industry','Name']).count().reset_index()
counts_df = counts_df.sort_values(by=['Industry','Year'],ascending=[True,False]).rename({'Year':'Freq'},axis=1)
counts_df.groupby(['Industry']).head(2)

,Industry,Name,Freq
2,Advertising & Marketing,Elite SEM,3
0,Advertising & Marketing,"4imprint, Inc.",2
10,Aerospace,"Stellar Solutions, Inc.",1
11,Aging Services,Care to Stay Home,1
12,Aging Services,ComForCare / At Your Side,1
15,Biotechnology & Pharmaceuticals,AbbVie,3
17,Biotechnology & Pharmaceuticals,Exact Sciences Corp,2
28,Construction,David Weekley Homes,6
34,Construction,Power Home Remodeling Group,5
42,Construction & Real Estate,Swinerton Incorporated,3


In [4]:
#What are the companies that made the Fortune list every year?
counts_df[counts_df['Freq']==6].sort_values(by='Name')

,Industry,Name,Freq
58,Financial Services & Insurance,Baird,6
321,Real Estate,Camden Property Trust,6
166,Information Technology,Cisco,6
28,Construction,David Weekley Homes,6
246,Manufacturing & Production,Hilcorp Energy Company,6
247,Manufacturing & Production,"JM Family Enterprises, Inc.",6
293,Professional Services,"Kimley-Horn and Associates, Inc",6
141,Hospitality,Kimpton Hotels & Restaurants,6
93,Financial Services & Insurance,Navy Federal Credit Union,6
336,Retail,Publix Super Markets,6


In [5]:
#Export data
writer = pd.ExcelWriter("BestCompanies_Millennials.xlsx")
companies_df.to_excel(writer,index=False) #export data as excel file
writer.save()